In [2]:
import os
import cv2
import json
import imageio
import numpy as np
import re
import colorsys
from collections import defaultdict

# ===== 改良版：各キーに対してより識別しやすい色を生成する =====
# グローバル変数でキーごとの色マッピングと次に使う色相を保持
_key_color_map = {}
_GOLDEN_RATIO_CONJUGATE = 0.618033988749895
_next_hue = 0.0

def get_color(key):
    """
    Golden ratio を用いて次々に異なる色相 (Hue) を生成し、
    各キーに一意の色を割り当てる。
    昨今のハッシュ法による画一的な色だけでなく、色相環上で均等に離れた色を生成できる。
    戻り値は OpenCV 用の BGR (0-255) タプル。
    """
    global _next_hue
    # まだ割り当てられていないキーなら、新色を生成してマッピング
    if key not in _key_color_map:
        # ゴールデンレシオ共役を加算して色相を決定
        hue = (_next_hue + _GOLDEN_RATIO_CONJUGATE) % 1.0
        _next_hue = hue
        s, v = 0.8, 0.9
        r, g, b = colorsys.hsv_to_rgb(hue, s, v)
        # BGR形式に変換して保存
        _key_color_map[key] = (int(b * 255), int(g * 255), int(r * 255))
    return _key_color_map[key]

# ========== ② 各種パスと対象フレーム範囲 ==========
# 対象動画ファイル（この動画は最初から始まるのでオフセット不要）
video_path = os.path.join("../../videos", "Drone", "1_1_6000.mp4")

# JSON（姿勢キーポイント）ファイルのディレクトリ
pose_dir = os.path.join("../../ground_truth", "Drone", "pose")

# MOT形式のbboxファイルのパス（MOT形式：各行 "frame id x y width height ..." を想定）
bbox_file_path = os.path.join("../../ground_truth", "Drone", "40_1215_gt.txt")

# 対象フレーム番号（動画とJSON, MOTが同じ番号付けになっていると仮定）
target_frame_start = 951
target_frame_end   = 1000

# bboxのタイミングがずれている場合のオフセット補正（今回は逆にずれているので、MOT情報が後ろの場合）
# ここでは「bbox_offset = 50」として、現在のフレーム番号から50フレーム引いた番号のbboxを使用する
bbox_offset = 40

# ========== ③ MOT形式のbboxファイルの読み込み ==========
# bbox_lines_frame: キー＝フレーム番号, 値＝該当フレームの検出リスト（各検出は (id, x, y, width, height)）
bbox_lines_frame = defaultdict(list)
if os.path.exists(bbox_file_path):
    with open(bbox_file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue
            try:
                frame_no = int(parts[0])
                det_id   = int(parts[1])
                x = float(parts[2])
                y = float(parts[3])
                width  = float(parts[4])
                height = float(parts[5])
                bbox_lines_frame[frame_no].append((det_id, x, y, width, height))
            except Exception as e:
                print("bbox parse error:", e)
else:
    print("bboxファイルが見つかりません:", bbox_file_path)

# ========== ④ 動画ファイルの準備 ==========
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"動画ファイル {video_path} を開けません。")

# 対象フレームは番号951～1000（動画内のフレーム番号と対応）
current_frame = 0
annotated_frames = []

# ========== ⑤ 各フレームごとの処理 ==========
while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame += 1
    if current_frame < target_frame_start or current_frame > target_frame_end:
        continue

    # 対応するJSONファイル（例："frame_951.json"）を読み込む
    json_filename = f"frame_{current_frame}.json"
    json_path = os.path.join(pose_dir, json_filename)
    if not os.path.exists(json_path):
        print(f"JSONファイルが見つかりません: {json_path}")
        continue
    with open(json_path, 'r') as f:
        data = json.load(f)

    # (A) 各キーごとにそのフレームの姿勢キーポイント群の重心を計算
    key_centroids = {}
    annotations = data.get("annotations", {})
    for key, points in annotations.items():
        pts_arr = np.array(points)
        if pts_arr.size == 0:
            continue
        centroid = np.mean(pts_arr, axis=0)
        key_centroids[key] = centroid

    # (B) 当該フレームにおけるMOT検出（bbox）の取得（オフセット補正：現在のフレーム番号から50を引く）
    available_dets = []
    corrected_frame = current_frame - bbox_offset
    if corrected_frame in bbox_lines_frame:
        for det in bbox_lines_frame[corrected_frame]:
            # 各検出の中心座標 (x + width/2, y + height/2)
            _, x, y, width, height = det
            center = np.array([x + width/2, y + height/2])
            available_dets.append({'det': det, 'center': center})
    else:
        available_dets = []
    
    # (C) 各キーごとに、未割当の検出から重心との距離が最小のものをグリーディーに割当
    key_to_bbox = {}
    for key, cent in key_centroids.items():
        best_idx = -1
        best_dist = float('inf')
        for idx, det_info in enumerate(available_dets):
            d = np.linalg.norm(cent - det_info['center'])
            if d < best_dist:
                best_dist = d
                best_idx = idx
        if best_idx != -1:
            key_to_bbox[key] = available_dets[best_idx]['det']
            available_dets.pop(best_idx)
    
    # (D) 描画: キーポイントを円で描画（線は結ばない）
    for key, points in annotations.items():
        color = get_color(key)
        radius = 8
        for p in points:
            pt = (int(round(p[0])), int(round(p[1])))
            cv2.circle(frame, pt, radius, color, thickness=-1)
    
    # (E) 描画: 対応付けたbboxをキーごとの色で矩形描画
    for key, det in key_to_bbox.items():
        _, x, y, width, height = det
        top_left = (int(round(x)), int(round(y)))
        bottom_right = (int(round(x + width)), int(round(y + height)))
        color = get_color(key)
        cv2.rectangle(frame, top_left, bottom_right, color, thickness=2)

    # (F) フレームをRGB形式に変換してリストに追加
    annotated_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    annotated_frames.append(annotated_frame)

cap.release()

# ========== ⑥ GIFおよびMP4の出力 ==========
if annotated_frames:
    fps = 20  # フレームレート（調整可能）

    # GIF出力
    # gif_out_path = 'annotated_animation_frames_951-1000.gif'
    # imageio.mimsave(gif_out_path, annotated_frames, fps=fps)
    # print(f"GIF画像を保存しました: {gif_out_path}")

    # MP4出力
    height, width, _ = annotated_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_out_path = 'annotated_video_frames_951-1000.mp4'
    video_writer = cv2.VideoWriter(video_out_path, fourcc, fps, (width, height))
    for frame in annotated_frames:
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        video_writer.write(frame_bgr)
    video_writer.release()
    print(f"MP4動画を保存しました: {video_out_path}")
else:
    print("対象フレームがないか、画像の読み込みに失敗しました。")


MP4動画を保存しました: annotated_video_frames_951-1000.mp4
